In [119]:
import nltk
nltk.download('punkt')
nltk.download('gutenberg')
import pandas as pd
from google.colab import drive 
drive.mount('/content/gdrive')
import os 
from nltk.tokenize import sent_tokenize, word_tokenize

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option("max_colwidth", 100)

import time
import re
import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [120]:
def normalize_text(s):
  import string, re
  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ",text)

  def white_space_fix(text):
    return " ".join(text.split())
    
  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    text = (word.lower() for word in text)
    return text 

  return white_space_fix(remove_articles(remove_punc(lower(s))))

In [121]:
def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()

  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)

  if len(common_tokens) == 0:
    return 0

  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  '''
  print('pred_tokens:' , pred_tokens)
  print('truth_tokens:', truth_tokens)
  print('prec:', prec)
  print('rec:', rec)
  '''

  
  return 2 * (prec * rec) / (prec + rec)

In [122]:
df = pd.read_csv('gdrive/My Drive/Colab Notebooks/CRF/data/49_df_quote_detection_0_8670.csv')
df.rename(columns = {'Unnamed: 0':'sent_id', 'QUOTE2':'quote'}, inplace = True)
df

,sent_id,quote,maximum,10sents_span
0,0,I am so clever that sometimes I don't understand a single word of what I am saying.,0.349206,"“You are a very irritating person,” said the Rocket, “and very ill-bred. I hate people who talk ..."
1,1,"Without music, life would be a mistake.",1.000000,--A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man al...
2,2,"The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...",0.571429,"She and your brother choose to go, and you will be only getting ill will.” Catherine submitted,..."
3,3,"The fool doth think he is wise, but the wise man knows himself to be a fool.",0.347826,"It is a task for Hercules.” “I will help you,” said Leonato, “if I have to sit up ten nights.” ..."
4,4,"Love all, trust a few, do wrong to none.",0.363636,"DOUBTS AND FEARS. I am cabin'd, cribb'd, confined, bound in To saucy doubts and fears. Macbe..."
...,...,...,...,...
8551,8664,"I am glad,"" he said, ""that I do not dwell in your country among such savage peoples. Here, in Ca...",0.241135,"""You did doubt me,"" she replied in a level voice. ""You practically accused me of aiding Baron vo..."
8552,8665,…he felt he could willingly give his life for them.,0.275862,"I tell you I didn't I didn't!” cried Nat, in a desperate sort of way. “Gently, gently, my son! W..."
8553,8666,"…growing pale and sober with the thought that her fate was soon to be decided; for, like all you...",0.246914,"He held the knife in his hand, admiring and longing for it, till the man packed up his goods to ..."
8554,8668,"If you feel your value lies in being merely decorative, I fear that someday you might find yours...",0.229730,"“We are playing nargerie,” explained Kitty. “Dis is mine effalunt!” added Tina, holding on by th..."


In [123]:
def clean_col(df, col):
  df[col] = df[col].replace("\n", ' ', regex=True)
  #df[col] = df[col].replace("—", ' ', regex=True)
  #df[col] = df[col].replace('_', ' ', regex=True)
  #df[col] = df[col].replace('-', '', regex=True)
  df[col] = df[col].replace("\[", '', regex=True)
  df[col] = df[col].replace("\]", '', regex=True)
  df[col] = df[col].replace("“", '', regex=True)
  df[col] = df[col].replace("”", '', regex=True)
  df[col] = df[col].replace('"', '', regex=True) 
  #df[col] = df[col].replace("’", '', regex=True)
  #df[col] = df[col].replace("'", '', regex=True)
  df[col] = df[col].replace("è", 'e', regex=True)
  df[col] = df[col].replace("é", 'e', regex=True)
  #df[col] = df[col].replace(";", '', regex=True) 
  df[col] = df[col].replace(' de ', ' ', regex=True)
  df[col] = df[col].replace('(?:\s+\*)+', '', regex=True)
  df[col] = df[col].replace('(?:\s+\.)+', '.', regex=True)
  df[col] = df[col].replace('\.{2,}', '.', regex=True)
  df[col] = df[col].replace('\s{2,}', ' ', regex=True)
  df[col] = df[col].replace('\-{2,}', ' ', regex=True)
  df[col] = df[col].replace('\&{2,}', ' ', regex=True)
  df[col] = df[col].replace('\.\?', '?', regex=True)

  return df

df = clean_col(df, 'quote')
df = clean_col(df, '10sents_span')

In [124]:
df = df[df['maximum'] >= 0.50].reset_index(drop=True)
df

,sent_id,quote,maximum,10sents_span
0,1,"Without music, life would be a mistake.",1.000000,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...
1,2,"The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...",0.571429,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a..."
2,5,"Love looks not with the eyes, but with the mind, And therefore is winged Cupid painted blind.",0.866667,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys..."
3,6,There is nothing I would not do for those who are really my friends. I have no notion of loving ...,0.857143,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget..."
4,7,I declare after all there is no enjoyment like reading! How much sooner one tires of any thing t...,0.750000,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ..."
...,...,...,...,...
5010,8657,"There are two sides to the life of every man, his individual life, which is the more free the mo...",0.683168,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...
5011,8658,"On the twelfth of June, the forces of Western Europe crossed the borders of Russia, and war bega...",0.631579,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin..."
5012,8660,"Imagine, if you can, a huge grizzly with ten legs armed with mighty talons and an enormous frogl...",0.809524,"There were no words, for they would have been a waste of breath. The very presence of the two pr..."
5013,8661,. smiles are the foundation of beauty.,0.500000,"They had anchored by night so as not to neglect a particle of the shore line, and it had happene..."


In [125]:
df['match']='-'
df['prev']='-'
df['next']='-'
df['prev_startid']=0
df['prev_endid']=0
df['match_startid']=0
df['match_endid']=0
df['next_startid']=0
df['next_endid']=0
df

,sent_id,quote,maximum,10sents_span,match,prev,next,prev_startid,prev_endid,match_startid,match_endid,next_startid,next_endid
0,1,"Without music, life would be a mistake.",1.000000,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,-,-,-,0,0,0,0,0,0
1,2,"The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...",0.571429,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...",-,-,-,0,0,0,0,0,0
2,5,"Love looks not with the eyes, but with the mind, And therefore is winged Cupid painted blind.",0.866667,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...",-,-,-,0,0,0,0,0,0
3,6,There is nothing I would not do for those who are really my friends. I have no notion of loving ...,0.857143,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",-,-,-,0,0,0,0,0,0
4,7,I declare after all there is no enjoyment like reading! How much sooner one tires of any thing t...,0.750000,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...",-,-,-,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5010,8657,"There are two sides to the life of every man, his individual life, which is the more free the mo...",0.683168,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,-,-,-,0,0,0,0,0,0
5011,8658,"On the twelfth of June, the forces of Western Europe crossed the borders of Russia, and war bega...",0.631579,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...",-,-,-,0,0,0,0,0,0
5012,8660,"Imagine, if you can, a huge grizzly with ten legs armed with mighty talons and an enormous frogl...",0.809524,"There were no words, for they would have been a waste of breath. The very presence of the two pr...",-,-,-,0,0,0,0,0,0
5013,8661,. smiles are the foundation of beauty.,0.500000,"They had anchored by night so as not to neglect a particle of the shore line, and it had happene...",-,-,-,0,0,0,0,0,0


In [126]:
for i in range(len(df)):    ### 5015
  quote = df['quote'][i]
  sents = nltk.sent_tokenize(df['10sents_span'][i])
  sents_text = df['10sents_span'][i]
  if i < 3: # just for check 3 samples
    print('span  ->', sents_text)

  maximum = 0
  j = 0
  quote_sent_len = len(nltk.sent_tokenize(quote))


  for sent_id in range(len(sents)-quote_sent_len):
    batch = sents[sent_id:sent_id+quote_sent_len]
    batch_joined = " ".join(batch)
    f1_score = compute_f1(batch_joined, quote)

    if f1_score > maximum:
      maximum = f1_score
      most_similar_sentence = batch_joined
    j = j+1
    
  most_similar_sentence_tokenized=nltk.sent_tokenize(most_similar_sentence)

  try:
    match_object = re.search(most_similar_sentence, sents_text)
    if i < 3: # just for check 3 samples
      print('quote ->', quote)
      print('match ->', most_similar_sentence)
      print()
    if match_object != None:
      match = sents_text[match_object.start():match_object.end()]
      df.at[i, 'match'] = match
      df.at[i, 'match_startid'] = match_object.start()
      df.at[i, 'match_endid'] = match_object.start() + len(match)

      prev_startid = 0
      prev_endid = match_object.start()-1
      df.at[i, 'prev_startid'] = prev_startid
      df.at[i, 'prev_endid'] = prev_endid
      df.at[i, 'prev'] = df['10sents_span'][i][prev_startid:prev_endid]

      next_startid = match_object.end() + 1
      next_endid = len(df['10sents_span'][i])
      df.at[i, 'next_startid'] = next_startid
      df.at[i, 'next_endid'] = next_endid
      df.at[i, 'next'] = df['10sents_span'][i][next_startid:next_endid]
  except:
    pass

new_cols = ["sent_id","maximum","10sents_span","quote","match","prev","next","prev_startid","prev_endid","match_startid","match_endid","next_startid","next_endid"]
df = df.reindex(columns=new_cols)

span  ->  A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man always docs too much. And precisely on that account he commits a second error, and then he does too little. 31 The trodden worm curls up. This testifies to its caution. It thus reduces its chances of being trodden upon again. In the language of morality: Humility.  32 There is such a thing as a hatred of lies and dissimulation, which is the outcome of a delicate sense of humour; there is also the selfsame hatred but as the result of cowardice, in so far as falsehood is forbidden by Divine law. Too cowardly to lie. 33 What trifles constitute happiness! The sound of a bagpipe. Without music life would be a mistake. The German imagines even God as a songster. 34 _On ne peut penser et ecrire qu'assis_ (G. Flaubert). Here I have got you, you nihilist! A sedentary life is the real sin against the Holy Spirit. Only those thoughts that come by walking have any value. 35 There are times when we 

In [127]:
### In some cases, the first two words of the match, prev, or next are the same.
### We join them with a '-' to handle CRF tagger error.

#for col in ['match', 'prev', 'next']:
#  for i in range(len(df)):
#    words = df.at[i, col].split(' ')
#    if len(words) > 1 and words[0] == words[1]:
#        cleaned = words[0] + '-' + ' '.join(words[1:])
#        df.at[i, col] = cleaned

In [128]:
df

,sent_id,maximum,10sents_span,quote,match,prev,next,prev_startid,prev_endid,match_startid,match_endid,next_startid,next_endid
0,1,1.000000,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,"Without music, life would be a mistake.",Without music life would be a mistake.,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,The German imagines even God as a songster. 34 _On ne peut penser et ecrire qu'assis_ (G. Flaube...,0,673,674,712,713,1264
1,2,0.571429,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...","She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","I have read all Mrs. Radcliffe’s works, and most of them with great pleasure. The Mysteries of U...",0,1882,1883,1985,1986,3726
2,5,0.866667,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Love looks not with the eyes, but with the mind, And therefore is winged Cupid painted blind.","Love looks not with the eyes, but with the mind; And therefore is wing’d Cupid painted blind.","I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Nor hath love’s mind of any judgment taste. Wings, and no eyes, figure unheedy haste. And theref...",0,436,437,530,531,1243
3,6,0.857143,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",There is nothing I would not do for those who are really my friends. I have no notion of loving ...,There is nothing I would not do for those who are really my friends. I have no notion of loving ...,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",My attachments are always excessively strong. I told Captain Hunt at one of our assemblies this ...,0,1020,1021,1155,1156,2343
4,7,0.750000,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...",I declare after all there is no enjoyment like reading! How much sooner one tires of any thing t...,I declare after all there is no enjoyment like reading! How much sooner one tires of anything th...,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...","No one made any reply. She then yawned again, threw aside her book, and cast her eyes round the ...",0,1296,1297,1494,1495,3610
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5010,8657,0.683168,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"There are two sides to the life of every man, his individual life, which is the more free the mo...","There are two sides to the life of every man, his individual life, which is the more free the mo...",Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"The king’s heart is in the hands of the Lord. A king is history’s slave. History, that is, the u...",0,1806,1807,2480,2481,5028
5011,8658,0.631579,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","On the twelfth of June, the forces of Western Europe crossed the borders of Russia, and war bega...","On the twelfth of June, 1812, the forces of Western Europe crossed the Russian frontier and war ...","Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","Millions of men perpetrated against one another such innumerable crimes, frauds, treacheries, th...",0,1835,1836,2012,2013,4288
5012,8660,0.809524,"There were no words, for they would have been a waste of breath. The very presence of the two pr...",

In [129]:
drop1=df.index[df["prev_endid"] == 0].tolist()
drop2=df.index[df["match_startid"] == 0].tolist() 
drop3=df.index[df["match_endid"] == 0].tolist()
drop4=df.index[df["next_startid"] == 0].tolist()
drop5=df.index[df["next_endid"] == 0].tolist()
drop=drop1+drop2+drop3+drop4+drop5
df=df.drop(df.index[drop]).reset_index(drop=True)

In [130]:
df

,sent_id,maximum,10sents_span,quote,match,prev,next,prev_startid,prev_endid,match_startid,match_endid,next_startid,next_endid
0,1,1.000000,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,"Without music, life would be a mistake.",Without music life would be a mistake.,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,The German imagines even God as a songster. 34 _On ne peut penser et ecrire qu'assis_ (G. Flaube...,0,673,674,712,713,1264
1,2,0.571429,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...","She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","I have read all Mrs. Radcliffe’s works, and most of them with great pleasure. The Mysteries of U...",0,1882,1883,1985,1986,3726
2,5,0.866667,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Love looks not with the eyes, but with the mind, And therefore is winged Cupid painted blind.","Love looks not with the eyes, but with the mind; And therefore is wing’d Cupid painted blind.","I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Nor hath love’s mind of any judgment taste. Wings, and no eyes, figure unheedy haste. And theref...",0,436,437,530,531,1243
3,6,0.857143,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",There is nothing I would not do for those who are really my friends. I have no notion of loving ...,There is nothing I would not do for those who are really my friends. I have no notion of loving ...,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",My attachments are always excessively strong. I told Captain Hunt at one of our assemblies this ...,0,1020,1021,1155,1156,2343
4,7,0.750000,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...",I declare after all there is no enjoyment like reading! How much sooner one tires of any thing t...,I declare after all there is no enjoyment like reading! How much sooner one tires of anything th...,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...","No one made any reply. She then yawned again, threw aside her book, and cast her eyes round the ...",0,1296,1297,1494,1495,3610
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4492,8657,0.683168,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"There are two sides to the life of every man, his individual life, which is the more free the mo...","There are two sides to the life of every man, his individual life, which is the more free the mo...",Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"The king’s heart is in the hands of the Lord. A king is history’s slave. History, that is, the u...",0,1806,1807,2480,2481,5028
4493,8658,0.631579,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","On the twelfth of June, the forces of Western Europe crossed the borders of Russia, and war bega...","On the twelfth of June, 1812, the forces of Western Europe crossed the Russian frontier and war ...","Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","Millions of men perpetrated against one another such innumerable crimes, frauds, treacheries, th...",0,1835,1836,2012,2013,4288
4494,8660,0.809524,"There were no words, for they would have been a waste of breath. The very presence of the two pr...",

# FORMAT FOR CRF INPUT

In [131]:
######### antecedent = prev    consequent = match ##########
df_crf_prev = df[['10sents_span', 'prev', 'match', 'prev_startid', 'prev_endid',	'match_startid',	'match_endid']].reset_index()
df_crf_prev = df_crf_prev.rename(columns={'index':'sentenceID',
                                          '10sents_span': 'sentence',
                                          'prev': 'antecedent',
                                          'match':'consequent',
                                          'prev_startid':'antecedent_startid',
                                          'prev_endid': 'antecedent_endid',
                                          'match_startid': 'consequent_startid',
                                          'match_endid': 'consequent_endid'})
df_crf_prev['sentenceID'] = df_crf_prev['sentenceID'] + 1
df_crf_prev

,sentenceID,sentence,antecedent,consequent,antecedent_startid,antecedent_endid,consequent_startid,consequent_endid
0,1,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,Without music life would be a mistake.,0,673,674,712
1,2,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...",0,1882,1883,1985
2,3,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Love looks not with the eyes, but with the mind; And therefore is wing’d Cupid painted blind.",0,436,437,530
3,4,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...","How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",There is nothing I would not do for those who are really my friends. I have no notion of loving ...,0,1020,1021,1155
4,5,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...","He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...",I declare after all there is no enjoyment like reading! How much sooner one tires of anything th...,0,1296,1297,1494
...,...,...,...,...,...,...,...,...
4492,4493,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"There are two sides to the life of every man, his individual life, which is the more free the mo...",0,1806,1807,2480
4493,4494,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","On the twelfth of June, 1812, the forces of Western Europe crossed the Russian frontier and war ...",0,1835,1836,2012
4494,4495,"There were no words, for they would have been a waste of breath. The very presence of the two pr...","There were no words, for they would have been a waste of breath. The very presence of the two pr...","Imagine, if you can, a huge grizzly with ten legs armed with mighty talons and an enormous frogl...",0,1880,1881,2267
4495,4496,"They had anchored by night so as not to neglect a particle of the shore line, and it had happene...","They had anchored by night so as not to neglect a particle of the shore line, and it had happene...","For, though Tarzan of the Apes was a killer of men and of beasts, he killed as the hunter kills,...",0,1476,1477,1841


In [132]:
######### antecedent = match    consequent = next ##########
df_crf_next = df[['10sents_span', 'match', 'next', 'match_startid',	'match_endid', 'next_startid', 'next_endid']].reset_index()
df_crf_next = df_crf_next.rename(columns={'index':'sentenceID',
                                          '10sents_span': 'sentence',
                                          'match': 'antecedent',
                                          'next':'consequent',
                                          'match_startid':'antecedent_startid',
                                          'match_endid': 'antecedent_endid',
                                          'next_startid': 'consequent_startid',
                                          'next_endid': 'consequent_endid'})
df_crf_next['sentenceID'] = df_crf_next['sentenceID'] + 1
df_crf_next

,sentenceID,sentence,antecedent,consequent,antecedent_startid,antecedent_endid,consequent_startid,consequent_endid
0,1,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,Without music life would be a mistake.,The German imagines even God as a songster. 34 _On ne peut penser et ecrire qu'assis_ (G. Flaube...,674,712,713,1264
1,2,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...","I have read all Mrs. Radcliffe’s works, and most of them with great pleasure. The Mysteries of U...",1883,1985,1986,3726
2,3,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Love looks not with the eyes, but with the mind; And therefore is wing’d Cupid painted blind.","Nor hath love’s mind of any judgment taste. Wings, and no eyes, figure unheedy haste. And theref...",437,530,531,1243
3,4,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",There is nothing I would not do for those who are really my friends. I have no notion of loving ...,My attachments are always excessively strong. I told Captain Hunt at one of our assemblies this ...,1021,1155,1156,2343
4,5,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...",I declare after all there is no enjoyment like reading! How much sooner one tires of anything th...,"No one made any reply. She then yawned again, threw aside her book, and cast her eyes round the ...",1297,1494,1495,3610
...,...,...,...,...,...,...,...,...
4492,4493,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"There are two sides to the life of every man, his individual life, which is the more free the mo...","The king’s heart is in the hands of the Lord. A king is history’s slave. History, that is, the u...",1807,2480,2481,5028
4493,4494,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","On the twelfth of June, 1812, the forces of Western Europe crossed the Russian frontier and war ...","Millions of men perpetrated against one another such innumerable crimes, frauds, treacheries, th...",1836,2012,2013,4288
4494,4495,"There were no words, for they would have been a waste of breath. The very presence of the two pr...","Imagine, if you can, a huge grizzly with ten legs armed with mighty talons and an enormous frogl...",Before I could call him off he had crushed Lakor into a jelly with a single blow of one mighty p...,1881,2267,2268,4638
4495,4496,"They had anchored by night so as not to neglect a particle of the shore line, and it had happene...","For, though Tarzan of the Apes was a killer of men and of beasts, he killed as the hunter kills,...",One thing the girl had noticed particularly when she had seen Tarzan rushing upon Terkoz—the viv...,1477,1841,1842,3718


# MERGE PREV AND NEXT AS ANTECEDENT

In [133]:
df_crf_both = df.copy(deep=True)

df_crf_both['antecedent'] = df_crf_both['prev'] + ' ' + df_crf_both['next']
df_crf_both['consequent'] = df_crf_both['match']
df_crf_both['sentence'] = df_crf_both['antecedent'] + ' ' + df_crf_both['consequent']
df_crf_both['antecedent_startid'] = 0
df_crf_both['antecedent_endid'] = 0
df_crf_both['consequent_startid'] = 0
df_crf_both['consequent_endid'] = 0
for i in range(len(df_crf_both)):
  aLen = len(df_crf_both.at[i, 'antecedent'])
  cLen = len(df_crf_both.at[i, 'consequent'])
  df_crf_both.at[i, 'antecedent_endid'] = aLen
  df_crf_both.at[i, 'consequent_startid'] = aLen + 1
  df_crf_both.at[i, 'consequent_endid'] = aLen + 1 + cLen

df_crf_both = df_crf_both.reset_index()
df_crf_both = df_crf_both[['index', 'sentence', 'antecedent','consequent','antecedent_startid','antecedent_endid',	'consequent_startid','consequent_endid']]
df_crf_both = df_crf_both.rename(columns={'index':'sentenceID'})
df_crf_both['sentenceID'] = df_crf_both['sentenceID'] + 1
df_crf_both

,sentenceID,sentence,antecedent,consequent,antecedent_startid,antecedent_endid,consequent_startid,consequent_endid
0,1,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,A dentist's question. 30 Errors of haste are seldom committed singly. The first time a man alwa...,Without music life would be a mistake.,0,1225,1226,1264
1,2,"She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","She and your brother choose to go, and you will be only getting ill will. Catherine submitted, a...","The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably s...",0,3623,3624,3726
2,3,"I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","I will, my Hermia. _Exit Hermia._ Helena, adieu. As you on him, Demetrius dote on you! _Exit Lys...","Love looks not with the eyes, but with the mind; And therefore is wing’d Cupid painted blind.",0,1149,1150,1243
3,4,"How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...","How much I am obliged to you; and when you have finished Udolpho, we will read the Italian toget...",There is nothing I would not do for those who are really my friends. I have no notion of loving ...,0,2208,2209,2343
4,5,"He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...","He then sat down by her, and talked scarcely to anyone else. Elizabeth, at work in the opposite ...",I declare after all there is no enjoyment like reading! How much sooner one tires of anything th...,0,3412,3413,3610
...,...,...,...,...,...,...,...,...
4492,4493,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,Without each of these causes nothing could have happened. So all these causes—myriads of causes—...,"There are two sides to the life of every man, his individual life, which is the more free the mo...",0,4354,4355,5028
4493,4494,"Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","Home! said Pierre, and despite twenty-two degrees of frost Fahrenheit he threw open the bearskin...","On the twelfth of June, 1812, the forces of Western Europe crossed the Russian frontier and war ...",0,4111,4112,4288
4494,4495,"There were no words, for they would have been a waste of breath. The very presence of the two pr...","There were no words, for they would have been a waste of breath. The very presence of the two pr...","Imagine, if you can, a huge grizzly with ten legs armed with mighty talons and an enormous frogl...",0,4251,4252,4638
4495,4496,"They had anchored by night so as not to neglect a particle of the shore line, and it had happene...","They had anchored by night so as not to neglect a particle of the shore line, and it had happene...","For, though Tarzan of the Apes was a killer of men and of beasts, he killed as the hunter kills,...",0,3353,3354,3718


# RECHECK CORRECTNESS OF START/END IDs

In [134]:
print(df_crf_prev.at[1,'sentence'][0:673])
print(df_crf_prev.at[1,'sentence'][674:712])

She and your brother choose to go, and you will be only getting ill will. Catherine submitted, and though sorry to think that Isabella should be doing wrong, felt greatly relieved by Mr. Allen’s approbation of her own conduct, and truly rejoiced to be preserved by his advice from the danger of falling into such an error herself. Her escape from being one of the party to Clifton was now an escape indeed; for what would the Tilneys have thought of her, if she had broken her promise to them in order to do what was wrong in itself, if she had been guilty of one breach of propriety, only to enable her to be guilty of another? CHAPTER 14 The next morning was fair, and Ca
herine almost expected another attack 


In [135]:
print(df_crf_next.at[1,'sentence'][674:712])
print(df_crf_next.at[1,'sentence'][713:1264])

herine almost expected another attack 
rom the assembled party. With Mr. Allen to support her, she felt no dread of the event: but she would gladly be spared a contest, where victory itself was painful, and was heartily rejoiced therefore at neither seeing nor hearing anything of them. The Tilneys called for her at the appointed time; and no new difficulty arising, no sudden recollection, no unexpected summons, no impertinent intrusion to disconcert their measures, my heroine was most unnaturally able to fulfil her engagement, though it was made with the hero himself. They determined


In [136]:
print(df_crf_both.at[1,'sentence'][0:1148])
print(df_crf_both.at[1,'sentence'][1226:1264])

She and your brother choose to go, and you will be only getting ill will. Catherine submitted, and though sorry to think that Isabella should be doing wrong, felt greatly relieved by Mr. Allen’s approbation of her own conduct, and truly rejoiced to be preserved by his advice from the danger of falling into such an error herself. Her escape from being one of the party to Clifton was now an escape indeed; for what would the Tilneys have thought of her, if she had broken her promise to them in order to do what was wrong in itself, if she had been guilty of one breach of propriety, only to enable her to be guilty of another? CHAPTER 14 The next morning was fair, and Catherine almost expected another attack from the assembled party. With Mr. Allen to support her, she felt no dread of the event: but she would gladly be spared a contest, where victory itself was painful, and was heartily rejoiced therefore at neither seeing nor hearing anything of them. The Tilneys called for her at the appoi

# TRAIN/TEST SPLIT & EXPORT

In [137]:
df_crf_prev_train = df_crf_prev[:int(len(df_crf_prev)*0.8)]
df_crf_prev_test = df_crf_prev[int(len(df_crf_prev)*0.8)+1:]
df_crf_next_train = df_crf_next[:int(len(df_crf_next)*0.8)]
df_crf_next_test = df_crf_next[int(len(df_crf_next)*0.8)+1:]
df_crf_both_train = df_crf_both[:int(len(df_crf_both)*0.8)]
df_crf_both_test = df_crf_both[int(len(df_crf_both)*0.8)+1:]

In [138]:
df_crf_prev_train.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/task2prev_train.csv",index=False)
df_crf_prev_test.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/task2prev_test.csv",index=False)
df_crf_next_train.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/task2next_train.csv",index=False)
df_crf_next_test.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/task2next_test.csv",index=False)
df_crf_both_train.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/task2both_train.csv",index=False)
df_crf_both_test.to_csv("gdrive/My Drive/Colab Notebooks/CRF/data/task2both_test.csv",index=False)